In [3]:
from google.colab import files
files.upload()


Saving jigsaw-toxic-comment-train-google-es-cleaned.csv to jigsaw-toxic-comment-train-google-es-cleaned (1).csv
Saving jigsaw-toxic-comment-train-google-fr-cleaned.csv to jigsaw-toxic-comment-train-google-fr-cleaned (1).csv
Saving jigsaw-toxic-comment-train-google-ru-cleaned.csv to jigsaw-toxic-comment-train-google-ru-cleaned (1).csv


In [5]:
!pip install sentence_transformers
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:


languages = [ 'ru', 'es', 'fr']
dfs = []
for lang in languages:
    df = pd.read_csv(f"./jigsaw-toxic-comment-train-google-{lang}-cleaned.csv")
    df['language'] = lang
    dfs.append(df)

In [11]:


# Ensure all languages are covered for each id in the sample
def sample_matching_ids(df, sample_size=10000):
    ids = df['id'].unique()
    sampled_ids = np.random.choice(ids, sample_size, replace=False)
    sampled_df = df[df['id'].isin(sampled_ids)]
    return sampled_df

# Sample data ensuring all matching ids are included
sampled_df = sample_matching_ids(df)


# Set up the models
# Universal Sentence Encoder (USE)
#use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# BERT
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModel.from_pretrained(bert_model_name)

# SBERT
sbert_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# RoBERTa
roberta_model_name = "xlm-roberta-base"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModel.from_pretrained(roberta_model_name)

# mBERT (Multilingual BERT)
mbert_model_name = "bert-base-multilingual-cased"
mbert_tokenizer = AutoTokenizer.from_pretrained(mbert_model_name)
mbert_model = AutoModel.from_pretrained(mbert_model_name)


In [12]:

# Helper function to get embeddings from BERT, mBERT, and RoBERTa
def get_transformer_embeddings(texts, tokenizer, model, max_length=128, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Generate embeddings for the comments
sampled_texts = sampled_df['comment_text'].tolist()

#+use_embeddings = use_model(sampled_texts).numpy()
bert_embeddings = get_transformer_embeddings(sampled_texts, bert_tokenizer, bert_model)
sbert_embeddings = sbert_model.encode(sampled_texts)
roberta_embeddings = get_transformer_embeddings(sampled_texts, roberta_tokenizer, roberta_model)
mbert_embeddings = get_transformer_embeddings(sampled_texts, mbert_tokenizer, mbert_model)
# gpt3_embeddings = np.array([get_gpt3_embeddings(text) for text in sampled_texts])

# Group comments by id
grouped = sampled_df.groupby('id')

# Function to compute similarity for each group
def compute_group_similarity(group, embeddings, index_mapping):
    indices = group.index.map(index_mapping).tolist()
    group_embeddings = embeddings[indices]
    sim_matrix = cosine_similarity(group_embeddings)
    return sim_matrix

# Create an index mapping from the DataFrame to embeddings
index_mapping = {idx: i for i, idx in enumerate(sampled_df.index)}

# Compute similarity for each model
bert_similarities = []
sbert_similarities = []
roberta_similarities = []
mbert_similarities = []
use_similarities = []

for group in grouped:
    bert_similarities.append(compute_group_similarity(group[1], bert_embeddings, index_mapping))
    sbert_similarities.append(compute_group_similarity(group[1], sbert_embeddings, index_mapping))
    roberta_similarities.append(compute_group_similarity(group[1], roberta_embeddings, index_mapping))
    mbert_similarities.append(compute_group_similarity(group[1], mbert_embeddings, index_mapping))
   # use_similarities.append(compute_group_similarity(group[1], use_embeddings, index_mapping))


# Print or analyze similarity results
# print("USE Similarity Matrices:")
# for sim in use_similarities:
#     print(sim)

print("BERT Similarity Matrices:")
for sim in bert_similarities:
    print(sim)

print("SBERT Similarity Matrices:")
for sim in sbert_similarities:
    print(sim)

print("RoBERTa Similarity Matrices:")
for sim in roberta_similarities:
    print(sim)

print("mBERT Similarity Matrices:")
for sim in mbert_similarities:
    print(sim)

# get averages of the similarity matrices to the 5th decimal place
bert_similarities_avg = np.mean(bert_similarities, axis=0)
sbert_similarities_avg = np.mean(sbert_similarities, axis=0)
roberta_similarities_avg = np.mean(roberta_similarities, axis=0)
mbert_similarities_avg = np.mean(mbert_similarities, axis=0)
use_similarities_avg = np.mean(use_similarities, axis=0)

print("BERT Average Similarity Matrix:")
print(bert_similarities_avg)

print("SBERT Average Similarity Matrix:")
print(sbert_similarities_avg)

print("RoBERTa Average Similarity Matrix:")
print(roberta_similarities_avg)

print("mBERT Average Similarity Matrix:")
print(mbert_similarities_avg)

# print("USE Average Similarity Matrix:")
# print(use_similarities_avg)


Streaming output truncated to the last 5000 lines.
[[1.]]
[[0.99999994]]
[[0.99999994]]
[[1.]]
[[1.]]
[[1.0000002]]
[[1.]]
[[1.0000002]]
[[0.9999999]]
[[1.]]
[[1.0000004]]
[[0.9999999]]
[[1.]]
[[1.0000002]]
[[1.0000001]]
[[0.9999998]]
[[1.]]
[[1.]]
[[0.99999994]]
[[1.]]
[[0.99999994]]
[[0.9999999]]
[[0.99999994]]
[[1.]]
[[1.0000002]]
[[1.0000002]]
[[1.]]
[[0.9999999]]
[[0.9999999]]
[[1.0000001]]
[[0.9999999]]
[[1.0000001]]
[[0.99999976]]
[[1.0000001]]
[[1.0000001]]
[[0.9999999]]
[[0.9999999]]
[[1.]]
[[1.0000001]]
[[1.]]
[[0.9999998]]
[[1.]]
[[1.0000001]]
[[1.]]
[[0.9999999]]
[[0.99999994]]
[[0.9999998]]
[[1.]]
[[0.99999994]]
[[1.]]
[[1.]]
[[0.99999976]]
[[1.0000002]]
[[0.9999999]]
[[0.9999998]]
[[0.99999994]]
[[1.0000002]]
[[1.]]
[[1.]]
[[1.]]
[[1.]]
[[1.0000001]]
[[1.0000001]]
[[1.0000001]]
[[0.9999999]]
[[1.]]
[[1.]]
[[0.9999998]]
[[0.9999998]]
[[0.9999999]]
[[1.0000002]]
[[0.9999999]]
[[0.9999999]]
[[1.]]
[[1.0000002]]
[[1.0000002]]
[[1.0000002]]
[[0.9999999]]
[[1.]]
[[1.0000001]]
[

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
